## What is does

Typical evaluation tools for binary models and data transformation for event series.

## Install package

In [2]:
%%capture
!pip3 install git+git://github.com/gabriela-plantie/evaluation_tools
    

In [3]:
from evaluation_tools import prediccion as pre

## Required packages

## Create example data prediction

In [4]:
import pandas as pd
import numpy as np
import scipy as sc
from scipy.stats import ks_2samp

In [5]:
size=10000
ds= pd.date_range('2019-01-01', periods=30, freq='M')
d = np.random.choice(ds, size, replace=True)
x1=np.random.uniform(0,1,size)
x2=np.random.uniform(0,1,size)
x3=np.random.uniform(0,1,size)
x4=np.random.uniform(0,1,size)/5

z = 1 + 2*x1 -4* x2 + 2*x2*x3 + x4
pr = 1/(1+np.exp(-z))
#l=list(pr[(pr>0.94) & ((pr<0.95))])
#l.sort()
#l

#pr[pr>0.95]=1
ide = np.arange(0, size,1)
y=sc.stats.binom.rvs(1, pr, size=size)
df = pd.DataFrame(data={'ide':ide, 'y':y, 'x1':x1, 'x2':x2, 'x3':x3, 'x4':x4, 'pr':pr, 'date': d})

#set(df.date)

In [7]:
min(df.pr)

0.05985366607848398

## Apply tools

In [6]:
p=pre.Predictor(df, 'ide', 'pr', 'y', 'date')
p.performance_table(q_group=[100,300])

media: 0.6176
events: 6176


,ide,y,br,pc_pob,pc_y,lift
pr_cut,,,,,,
"(-0.001, 0.298]",1000,210,0.210,0.10,0.03,0.3
"(0.298, 0.409]",1000,356,0.356,0.10,0.06,0.6
"(0.409, 0.501]",1000,458,0.458,0.10,0.07,0.7
"(0.501, 0.586]",1000,541,0.541,0.10,0.09,0.9
"(0.586, 0.659]",1000,608,0.608,0.10,0.10,1.0
"(0.659, 0.72]",1000,704,0.704,0.10,0.11,1.1
"(0.72, 0.776]",1000,727,0.727,0.10,0.12,1.2
"(0.776, 0.83]",1000,800,0.800,0.10,0.13,1.3
"(0.83, 0.883]",1000,850,0.850,0.10,0.14,1.4


In [ ]:
p.graph_ks()

In [ ]:
p.stability()
p.graph_stability()

In [ ]:
p.metricas_performance()

## Create example data events


In [ ]:

def f_to_dt(fecha_hora_min):
    return pd.to_datetime(fecha_hora_min, format='%Y-%m-%d %H:%M')
df= pd.DataFrame()
df['ide_instalacion']=[11,11,11,11,11,11,22,22,22,22]
cli=['2019-08-01 15:30', '2019-08-01 15:40', '2019-08-01 16:30', '2019-08-01 16:31','2019-08-01 20:30','2019-08-01 21:00']
cli2=[ '2019-08-01 15:00', '2019-08-02 10:30', '2019-08-02 15:30', '2019-08-03 15:30']
cli.extend(cli2)
df['fecha']= list(map(f_to_dt, cli))
df['evento']=['A', np.nan, 'D', 'A', 'D', np.nan,'A', 'D', 'D',  'A']
display(df)

## Apply tools
    

In [ ]:

e = pre.Events(df, grouper='ide_instalacion',  datetime_name='fecha', event_name='evento', every_x_minutes=15)

ide=22
display(e.df[e.df.ide_instalacion==ide][:5])
display(e.df[e.df.ide_instalacion==ide][-5:])
a = e.plot_ide(ide_value=ide, event_value_pos='A', figsize=(16,4))


In [ ]:
ide=11
display(e.df[e.df.ide_instalacion==ide][:5])
display(e.df[e.df.ide_instalacion==ide][-5:])
a = e.plot_ide(ide_value=ide, event_value_pos='A', figsize=(16,4))


In [ ]:
#jupyter nbconvert README.ipynb --to markdown